# Try_folium

In [1]:
# pip install folium

In [2]:
import json
import numpy as np
import pandas as pd
import datetime

import folium 
from folium.plugins import HeatMap , HeatMapWithTime



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def convertCoords(coord):
    # Convert E7 lat/long format to decimal.
    return coord / 1e7



with open('Records.json') as f:
    # Load in Google Location History data.
    data = json.load(f)

d = []

counter = 0

for i in data['locations']:
    # Loop through JSON, append useful data to list.
    d.append((convertCoords(i['latitudeE7']), convertCoords(i['longitudeE7']),i['serverTimestamp']))
#     print(counter)
#     print('lat-long : ',(i['latitudeE7'], i['longitudeE7']))
#     print('serverTimestamp : ',i['serverTimestamp'])
#     print('source : ', i['source'])
    counter += 1
    
    if not counter % 10000:
        # Print update every 10k entries (useful for large data).
        print("Counter: {}".format(counter))



def uniquify(seq):
    # Remove duplicate lat/long data.
    # Not order preserving.
    return list(set(seq))

# d = uniquify(d)

final = np.array([np.array(ele) for ele in d])

# print("------------")
# print("Unique lat/long entries remaining: {}".format(len(final)))
print("------------")
print("final : ",final)
print("------------")
df = pd.DataFrame(final, columns = ['latitude','longitude','datetime'])
df[['latitude','longitude']] = df[['latitude','longitude']].astype('float')

df['datetime'] = df['datetime'].apply(lambda x : x[0:10])

df

------------
final :  [['22.2801272' '114.1721057' '2022-12-06T04:24:38.218Z']
 ['22.2804285' '114.1721098' '2022-12-06T04:24:38.218Z']
 ['22.2801857' '114.1721118' '2022-12-06T04:24:38.218Z']
 ...
 ['22.2801156' '114.1721361' '2022-12-12T01:11:20.051Z']
 ['22.2802729' '114.172035' '2022-12-12T01:11:20.051Z']
 ['22.280221' '114.1720714' '2022-12-12T01:11:20.051Z']]
------------


,latitude,longitude,datetime
0,22.280127,114.172106,2022-12-06
1,22.280428,114.172110,2022-12-06
2,22.280186,114.172112,2022-12-06
3,22.280228,114.172038,2022-12-06
4,22.280256,114.172019,2022-12-06
...,...,...,...
2519,22.280214,114.171969,2022-12-12
2520,22.280285,114.172113,2022-12-12
2521,22.280116,114.172136,2022-12-12
2522,22.280273,114.172035,2022-12-12


# over all 

In [4]:
map_center = df[['latitude','longitude']].mean()
print('map_center :',list(map_center))

# 建立地圖與設定位置
fmap = folium.Map(location=map_center, zoom_start=13)

# fmap  # 在notebook中顯示地圖

polt_data = final
fmap.add_child(HeatMap(data=df[['latitude','longitude']]))

map_center : [22.304860893858987, 114.19910042274225]


In [5]:
fmap.save('index.html')

# By time

In [6]:
time_index = df['datetime'].sort_values().astype('str').unique()
time_index

array(['2022-12-06', '2022-12-07', '2022-12-08', '2022-12-09',
       '2022-12-10', '2022-12-11', '2022-12-12'], dtype=object)

In [7]:
data_by_time = []
for d in time_index:
    data_by_time.append(df[df['datetime'] == d][['latitude','longitude']])

data_by_time

[      latitude   longitude
 0    22.280127  114.172106
 1    22.280428  114.172110
 2    22.280186  114.172112
 3    22.280228  114.172038
 4    22.280256  114.172019
 ..         ...         ...
 347  22.320623  114.257570
 348  22.320623  114.257570
 349  22.320623  114.257570
 350  22.320623  114.257570
 351  22.320623  114.257570
 
 [352 rows x 2 columns],
       latitude   longitude
 352  22.320623  114.257570
 353  22.320623  114.257570
 354  22.320623  114.257570
 355  22.320623  114.257570
 356  22.320635  114.257559
 ..         ...         ...
 823  22.320593  114.257578
 824  22.320593  114.257578
 825  22.320593  114.257578
 826  22.320593  114.257578
 827  22.320479  114.257696
 
 [476 rows x 2 columns],
        latitude   longitude
 828   22.320479  114.257696
 829   22.320479  114.257696
 830   22.320479  114.257696
 831   22.320492  114.257683
 832   22.320492  114.257683
 ...         ...         ...
 1279  22.320643  114.257566
 1280  22.320643  114.257566
 1281  22.320

In [8]:
map_center = df[['latitude','longitude']].mean()
print('map_center :',list(map_center))

hmt = folium.Map(location=map_center, zoom_start=13)

# HeatMapWithTime(data_by_time,
#                 index=time_index,
#                ).add_to(hmt)

hmt

map_center : [22.304860893858987, 114.19910042274225]


In [9]:
# hmt.save('hmt.html')